In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn=psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=semawit password=semawit")
cur=conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files


In [7]:
#Process song_data

In [4]:
song_files=get_files('data/song_data')

In [6]:
filepath = song_files[0]

In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARDR4AC1187FB371A1,NaN,NaN,,Montserrat Caballé;Placido Domingo;Vicente Sar...,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,511.16363,0


In [11]:
#Extract Data for Songs Table

In [8]:
df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0]

array(['SOBAYLL12A8C138AF9', 'Sono andati? Fingevo di dormire',
       'ARDR4AC1187FB371A1', 0, 511.16363], dtype=object)

In [9]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOBAYLL12A8C138AF9',
 'Sono andati? Fingevo di dormire',
 'ARDR4AC1187FB371A1',
 0,
 511.16363]

Insert Record into Song Table

Implement the song_table_insert query in sql_queries.py and run the cell below to insert a record for this song into the songs table. Remember to run create_tables.py before running the cell below to ensure you've created/resetted the songs table in the sparkify database.

In [10]:
cur.execute(song_table_insert, song_data)
conn.commit()

In [11]:

#2: artists Table
#Extract Data for Artists Table

In [12]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data


['ARDR4AC1187FB371A1',
 'Montserrat Caballé;Placido Domingo;Vicente Sardinero;Judith Blegen;Sherrill Milnes;Georg Solti',
 '',
 nan,
 nan]

In [14]:
cur.execute(artist_table_insert, song_data)
conn.commit()

In [15]:

#Process log_data


Process log_data

In this part, you'll perform ETL on the second dataset, log_data, to create the time and users dimensional tables, as well as the songplays fact table.

Let's perform ETL on a single log file and load a single record into each table.

In [16]:
log_files = get_files('data/log_data')

In [21]:
filepath = log_files[0]

In [22]:
filepath

'/home/semawit/Desktop/files/projects/Data Modeling for a Music Streaming App/data/log_data/2018/11/2018-11-24-events.json'

In [24]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Future Sound of London,Logged In,Ryan,M,0,Smith,405.28934,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Papua New Guinea,200,1543020300796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,Harmonia,Logged In,Ryan,M,1,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Sehr kosmisch,200,1543020705796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
2,None,Logged Out,None,None,0,None,NaN,free,None,GET,Home,NaN,86,None,200,1543026352796,None,
3,None,Logged Out,None,None,1,None,NaN,free,None,GET,Home,NaN,86,None,200,1543026402796,None,
4,None,Logged Out,None,None,2,None,NaN,free,None,GET,Help,NaN,86,None,200,1543026454796,None,


In [25]:
df = df[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Future Sound of London,Logged In,Ryan,M,0,Smith,405.28934,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Papua New Guinea,200,1543020300796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,Harmonia,Logged In,Ryan,M,1,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Sehr kosmisch,200,1543020705796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
9,My Chemical Romance,Logged In,Dustin,M,7,Lee,311.11791,free,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",PUT,NextSong,1.540853e+12,86,Welcome To The Black Parade (Album Version),200,1543026602796,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,87
10,Disturbing Tha Peace / Ludacris / Mystikal / I-20,Logged In,Mohammad,M,0,Rodriguez,276.42730,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,888,Move Bitch,200,1543027714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
11,Carlos PaiÃÂ£o,Logged In,Mohammad,M,1,Rodriguez,185.99138,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,888,Playback,200,1543027990796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88


In [26]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0    2018-11-24 00:45:00.796
1    2018-11-24 00:51:45.796
9    2018-11-24 02:30:02.796
10   2018-11-24 02:48:34.796
11   2018-11-24 02:53:10.796
Name: ts, dtype: datetime64[ns]

In [27]:
time_data = (df['ts'].values, t.dt.hour.values, t.dt.day.values, t.dt.week.values, t.dt.month.values, t.dt.year.values, t.dt.weekday.values)
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']


/tmp/ipykernel_27397/554136775.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  time_data = (df['ts'].values, t.dt.hour.values, t.dt.day.values, t.dt.week.values, t.dt.month.values, t.dt.year.values, t.dt.weekday.values)


In [28]:
time_df = pd.DataFrame(data=list(zip(*time_data)), columns=column_labels)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1543020300796,0,24,47,11,2018,5
1,1543020705796,0,24,47,11,2018,5
2,1543026602796,2,24,47,11,2018,5
3,1543027714796,2,24,47,11,2018,5
4,1543027990796,2,24,47,11,2018,5


In [29]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [30]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [31]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

In [36]:
for i, row in user_df.iterrows():
    try:
        cur.execute(user_table_insert, row)
    except psycopg2.IntegrityError:
        pass
    conn.commit()

SyntaxError: syntax error at or near "SET"
LINE 10: ON CONFLICT (user_id) DO SET level = EXCLUDED.level;
                                  ^


In [37]:
df.head()


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Future Sound of London,Logged In,Ryan,M,0,Smith,405.28934,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Papua New Guinea,200,1543020300796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
1,Harmonia,Logged In,Ryan,M,1,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,873,Sehr kosmisch,200,1543020705796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
9,My Chemical Romance,Logged In,Dustin,M,7,Lee,311.11791,free,"Myrtle Beach-Conway-North Myrtle Beach, SC-NC",PUT,NextSong,1.540853e+12,86,Welcome To The Black Parade (Album Version),200,1543026602796,Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7....,87
10,Disturbing Tha Peace / Ludacris / Mystikal / I-20,Logged In,Mohammad,M,0,Rodriguez,276.42730,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,888,Move Bitch,200,1543027714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
11,Carlos PaiÃÂ£o,Logged In,Mohammad,M,1,Rodriguez,185.99138,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,888,Playback,200,1543027990796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88


In [38]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
